In [19]:
import numpy as np
import cvxpy as cp
import pandas as pd

type = 2  # Change this to 1 for HPWH
if type == 1:
    df = pd.read_csv("output_site_null.csv")

else:
    df = pd.read_csv("output_site_null_ewh.csv")

# Constants
cp_val = 4181.3
rho = 1000
dt = 8 #what should this be? euler steps per timestep?
T_a = 22
V = 250 #(L)

# Data
T_u = [value if mode == "Upper On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
T_m = [value if mode == "Lower On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
T_l = df["T_WH12"].values

W_list = []
z_list = []

for j in range(len(T_u) - 1):  # because of j+1
    Wj = np.zeros((3, 7))  # Each Wj is 3x7

    # Row 1: Upper node (δtpu)
    Wj[0, 2] = dt * (T_u[j] - T_a)       # U_u
    Wj[0, 4] = dt * (T_u[j] - T_m[j])    # K_um
    Wj[0, 6] = rho * cp_val * (T_u[j+1] - T_u[j])  # V_u

    # Row 2: Middle node (δtpm)
    Wj[1, 1] = dt * (T_m[j] - T_a)       # U_m
    Wj[1, 3] = dt * (T_m[j] - T_l[j])    # K_ml
    Wj[1, 4] = dt * (T_m[j] - T_u[j])    # K_um
    Wj[1, 5] = rho * cp_val * (T_m[j+1] - T_m[j])  # V_m

    # Row 3: Lower node
    Wj[2, 0] = dt * (T_l[j] - T_a)       # U_l
    Wj[2, 3] = dt * (T_l[j] - T_m[j])    # K_ml
    Wj[2, 5] = -rho * cp_val * (T_l[j+1] - T_l[j])  # -V_m
    Wj[2, 6] = -rho * cp_val * (T_l[j+1] - T_l[j])  # -V_u

    # not sure what this should be
    z_j = np.array([
        # δtpu upper node
        dt * (T_u[j]), #Power node 3 * time interval (60s?)

        # δtpm middle node
        dt * (T_m[j]),

        # lower node energy change
        V * rho * cp_val * (T_l[j] - T_l[j+1]) #node 16 cold water input
    ])

    W_list.append(Wj)
    z_list.append(z_j)

# Stack based on time dimension
W = np.vstack(W_list)  # (3N x 7)
z_full = np.hstack(z_list)  # (3N, )

# CVXPY OLS problem
theta = cp.Variable(7) #Ul, Um, Uu, Kml, Kum, Vm, Vu] U - Tank Insulation, K - thermal conductivity, V - node volume
cost = cp.sum_squares(W @ theta - z_full) #z_j = W_j @ theta
problem = cp.Problem(cp.Minimize(cost))
problem.solve()

# Output estimated parameters
param_names = ["Ul", "Um", "Uu", "Kml", "Kum", "Vm", "Vu"]
print("\nThe optimal value is", problem.value)
for name, val in zip(param_names, theta.value):
    print(f"{name} = {val:.4f}")

print("The norm of the residual is ", cp.norm(W @ theta - z_full, p=2).value)



The optimal value is 4.4944717676283175e+19
Ul = 160761.3275
Um = 215272.2738
Uu = -12495.6420
Kml = -88845.5029
Kum = 17430289.4647
Vm = 44.1972
Vu = 198.5966
The norm of the residual is  6704082165.090394


In [4]:
W_list[0]

array([[ 0.00000000e+00,  0.00000000e+00, -1.76000000e+02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -1.76000000e+02,  0.00000000e+00,
        -3.86819350e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 2.10819350e+02,  0.00000000e+00,  0.00000000e+00,
         3.86819350e+02,  0.00000000e+00,  4.08005133e+05,
         4.08005133e+05]])

In [1]:
#Rearrange energy balance formulas to compute T[j + 1]

def predict_temperatures(T_init, timesteps, theta, draws, dt=1, cp=4181.3, rho=1000, T_a=22, V =250):
    Ul, Um, Uu, Kml, Kum, Vm, Vu = theta
    Vl = V - Vm  - Vu
    Tu_pred, Tm_pred, Tl_pred = [], [], []

    # Starting Temperatures [upper, middle, lower] nodes
    Tu_pred.append(T_init[0])
    Tm_pred.append(T_init[1])
    Tl_pred.append(T_init[2])

    for t in range(timesteps - 1):
        Tu = Tu_pred[-1] #get last predictions
        Tm = Tm_pred[-1]
        Tl = Tl_pred[-1]
        vt = draws[t] #draws

        #rearrange energy balance equations for Euler Update
        # Upper node
        Tu_next = Tu + (1 / (rho * cp * Vu)) * (
            Uu * dt * vt * (T_a - Tu) +
            Kum * dt * vt * (Tm - Tu)
        )

        # Middle node
        Tm_next = Tm + (1 / (rho * cp * Vm)) * (
            Um * dt * vt *  (T_a - Tm) +
            Kml * dt * vt * (Tl - Tm) +
            Kum * dt * vt * (Tu - Tm)
        )

        # Lower node
        Tl_next = Tl - (1 / (rho * cp * (V + Vm + Vu)))  *(
            Ul * dt * vt *  (Tl - T_a) +
            Kml * dt * vt * (Tl - Tm)
        )

        Tu_pred.append(Tu_next)
        Tm_pred.append(Tm_next)
        Tl_pred.append(Tl_next)

    return np.array(Tu_pred), np.array(Tm_pred), np.array(Tl_pred)

# Validation

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Choose data based on type

if type == 1:  # HPWH
    val_data = pd.read_csv("output_site_90023.csv")
else:  # EWH
    val_data = pd.read_csv("output_site_90023_ewh.csv")

# Parameters
window_size = 8
predictions = []
true_values = []

# Sanity check, iterate through idle
for index in range(len(val_data) - window_size):
    t_u = val_data['T_WH3'].iloc[index]
    t_m = val_data['T_WH10'].iloc[index]
    t_l = val_data['T_WH12'].iloc[index]
    draws = val_data['Draw Data'].iloc[index: index + window_size].reset_index(drop=True)
    
    y_pred = predict_temperatures([t_u, t_m, t_l], window_size, theta.value, draws)
    y_true = [
        val_data['T_WH3'].iloc[index  : index  + window_size].values,
        val_data['T_WH10'].iloc[index : index  + window_size].values,
        val_data['T_WH12'].iloc[index : index + window_size].values
    ]
    
    predictions.append(y_pred)
    true_values.append(y_true)
    index += 1

# Flatten lists for metrics, depending on model output format
# Example: compute MSE for each level
import numpy as np

y_pred_array = np.array(predictions)  # shape: (n_samples, 3, window_size) or similar
y_true_array = np.array(true_values)

mse_upper = mean_squared_error(y_true_array[:, 0, :].flatten(), y_pred_array[:, 0, :].flatten())
mse_middle = mean_squared_error(y_true_array[:, 1, :].flatten(), y_pred_array[:, 1, :].flatten())
mse_lower = mean_squared_error(y_true_array[:, 2, :].flatten(), y_pred_array[:, 2, :].flatten())

print(f"MSE Upper: {mse_upper:.3f}")
print(f"MSE Middle: {mse_middle:.3f}")
print(f"MSE Lower: {mse_lower:.3f}")


KeyError: 'Draw Data (L)'

In [ ]:
draws = val_data['Draw Data (L)'].iloc[index: index + window_size]
draws[0]

0.0

Experiments
1) Perfect foresight of future hourly average water use patterns (accurate in daily shape but not granular)
    36, 54, and 72 gal/day draw profilesout
    Simulation based test 28.8 to 72 gal/day

2) CasADi python optimization package, IPOPT solver

3) Use cvxpy for OLS, etc.